<a href="https://colab.research.google.com/github/Karim-Anwar/masterProject/blob/main/preporcess_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy
import datetime

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Autocorrelation
from statsmodels.graphics.tsaplots import plot_acf
# Partial Autocorrelation
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
import seaborn as sns

In [ ]:
from statsmodels.tsa.stattools import ccf

In [3]:
import os
import re

In [ ]:
def read_csv_files_from_directories(root_directory):
    directories = []
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if file.endswith('.csv'):
                directories.append(root)
                break
    return directories

In [ ]:
# def extract_numbers_from_csv_names(root_directory):
#     numbers = []
#     pattern = r'TL(\d+)\.csv'

#     for dirpath, dirnames, filenames in os.walk(root_directory):
#       # if dirpath == root_directory:
#       # # Skip the root directory
#       #   continue
#       for file_name in filenames:
#           if file_name.endswith('.csv'):
#               match = re.search(pattern, file_name)
#               if match:
#                   extracted_number = match.group(1)
#                   numbers.append(int(extracted_number))
    
#     return numbers

In [ ]:
def extract_numbers_from_csv_names(file_name):
    numbers = []
    pattern = r'TL(\d+)\.csv'
    
    match = re.search(pattern, file_name)
    if match:
        extracted_number = match.group(1)
        numbers.append(int(extracted_number))
    
    return numbers

In [ ]:
def extract_direction_from_subdirectories(root_directory):
    directions = []
    pattern = r'.*([ENSW])$'

    for dirpath, dirnames, filenames in os.walk(root_directory):
        for dirname in dirnames:
            match = re.search(pattern, dirname)
            if match:
                extracted_direction = match.group(1)
                directions.append(extracted_direction)

    return directions

In [ ]:
def get_fill_method(number):
    if number < 10:
        return 'ffill'  # Fill forward
    elif number % 10 == 1:
        return 'backfill'  # Backward fill
    elif number % 10 == 2:
        return 'interpolate'  # Linear interpolation
    else:
        return None

In [ ]:
def read_and_resample_csv_files(directory, start_index=None, end_index=None):
    dataframes = []
    file_names = os.listdir(directory)


    for file_name in file_names:
      if file_name.endswith('.csv'):
          file_path = os.path.join(directory, file_name)
          
          df = pd.read_csv(file_path)
          fill_method = get_fill_method(extract_numbers_from_csv_names(file_name)[0])
          
          if fill_method is None:
            return None
          elif fill_method == 'linear':
            return None

          df.columns = ['seq', 'time', 'value']

          # Drop the 'seq' column
          if 'seq' in df.columns:
              df.drop('seq', axis=1, inplace=True)
          
          df['time'] = pd.to_datetime(df['time'])
          df.set_index('time', inplace=True)
          
          df = df.loc[~df.index.duplicated(keep='first')]
          
          # Sort the index in ascending order
          df.sort_index(inplace=True)


          if fill_method == 'interpolate':
            resampled_df = df.asfreq('1S').interpolate(method='linear')
          else:
            resampled_df = df.asfreq('1S', method=fill_method)

          
          if start_index is not None and end_index is not None:
              resampled_df = resampled_df.loc[start_index:end_index]
            
      dataframes.append(resampled_df)
            
    if dataframes:
      merged_df = pd.concat(dataframes, axis=1)

      merged_df.columns = ['pir', 'shade', 'alight'] # Rename columns based on the number
      return merged_df
    
    return None

In [ ]:
root_directory = '/content/drive/MyDrive/exploratory-data-analysis/data/'
output_directory = '/content/drive/MyDrive/exploratory-data-analysis/cleaned_data'
# Step 1: Read CSV files from multiple directories
directories = read_csv_files_from_directories(root_directory)[1:]

# Step 2: Read and merge CSV files from the same directory
merged_dataframes = []
for directory in directories:
    merged_df = read_and_resample_csv_files(directory, start_index='2023-03-22 00:20:00', end_index='2023-05-19 12:10:00')
    if merged_df is not None:
        merged_dataframes.append(merged_df)

for i, df in enumerate(merged_dataframes):
    original_directory = directories[i]
    directory_name = os.path.basename(original_directory)
    file_name = f'{directory_name}.csv'
    file_path = os.path.join(output_directory, file_name)
    df.to_csv(file_path, index=True)

In [ ]:
merged_dataframes

In [49]:
# Directory paths
resampled_directory = '/content/drive/MyDrive/exploratory-data-analysis/data'
second_directory = '/content/drive/MyDrive/exploratory-data-analysis/cleaned_data_not'
output_directory = '/content/drive/MyDrive/exploratory-data-analysis/cleaned_data'


# Perform resampling and interpolation
start_index = '2023-03-22 00:20:00'
end_index = '2023-05-19 12:10:00'


# Read and resample the North, East, South, and West CSVs
def resample_csv(csv_path):
    df = pd.read_csv(csv_path, parse_dates=['time'], index_col='time', names=['seq', 'time', 'value'])
    df = df.drop(['seq'], axis=1)

    # Drop duplicate timestamps
    df = df.loc[~df.index.duplicated(keep='first')]
    # Sort the index in ascending order
    df.sort_index(inplace=True)
    
    # Resample and interpolate
    df = df.asfreq('1S').interpolate(method='linear').loc[start_index:end_index]
    return df

north_df = resample_csv(os.path.join(resampled_directory, 'North.csv'))
east_df = resample_csv(os.path.join(resampled_directory, 'East.csv'))
south_df = resample_csv(os.path.join(resampled_directory, 'South.csv'))
west_df = resample_csv(os.path.join(resampled_directory, 'West.csv'))

# Create a dictionary mapping direction to dataframe
direction_mapping = {'N': north_df, 'E': east_df, 'S': south_df, 'W': west_df}

# Iterate over the second directory CSVs
for file_name in os.listdir(second_directory):
    if file_name.endswith('.csv'):
        # Extract the direction from the file name
        direction = file_name.split('_')[-1][0]
        
         # Read the CSV file and set the 'time' column as index
        second_df = pd.read_csv(os.path.join(second_directory, file_name), parse_dates=['time'], index_col='time')
        
        # Merge with the corresponding resampled dataframe
        if direction in direction_mapping:
            merged_df =pd.merge(second_df, direction_mapping[direction], left_index=True, right_index=True)

            # Save the merged dataframe with the original filename
            output_filename = os.path.join(output_directory, os.path.splitext(file_name)[0] + '.csv')
            merged_df.to_csv(output_filename, index=True, index_label='time')  # Set index=True to include the 'time' index